In [ ]:
import pandas as pd
import requests
import time

def get_html(code: str, page: int) -> str:
    url = "https://finance.naver.com/item/sise_day.nhn?"  # 네이버 주식
    querystring = {"code":code, "page":page}
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
    response = requests.get(url, params=querystring, headers=headers)
    try:
        response.raise_for_status()
    except Exception as exc:
        print(exc)
        exit(0)
    
    return response.text

def scrap_data(code: str, sleep_time: int=2) -> pd.DataFrame:
    df = pd.DataFrame()
    last_page = 37  # 1페이지당 10일
    for page in range(1, last_page+1):
        date_converter = lambda d:d.replace('.', '-')
        dfs = pd.read_html(io=get_html(code, page), converters={0:date_converter})
        df = df.append(dfs[0].dropna().astype(dtype='int64', errors='ignore'), ignore_index=True)
        time.sleep(sleep_time)
    return df

def store_data(df: pd.DataFrame, code: int) -> None:
    df.sort_values(by='날짜', ascending=True, inplace=True)
    df.to_csv(f"{code}.csv", index=False)

code = '035420' # 종목 코드
store_data(scrap_data(code), code)